# MongoDB -  Consultas

Base de dados importada - xxxxxxx


## Importando biblioteca e conectando ao cliente

In [1]:
!pip install pymongo

In [99]:
# Imports iniciais
import pymongo

# Imports para ajudar na visualização
from pprint import pprint

# Conectando ao cliente local na port 27017
client = pymongo.MongoClient('localhost',27017)

In [100]:
db = client.mongo_db

In [102]:
cursor = db.doc_table.find({"database" : "boston"}).limit(5)

for doc in cursor:
    pprint(doc)

{'_id': ObjectId('6083b753779724d204c70a06'),
 'database': 'boston',
 'date': datetime.datetime(2021, 4, 24, 3, 14, 43, 126000),
 'name': 'SVR',
 'params': {'C': 0.1,
            'cache_size': 200,
            'coef0': 0.0,
            'degree': 2,
            'epsilon': 0.1,
            'gamma': 'auto_deprecated',
            'kernel': 'linear',
            'max_iter': -1,
            'shrinking': True,
            'tol': 0.001,
            'verbose': False},
 'scores': {'fit_time': 0.40048933029174805,
            'score_time': 0.0032573938369750977,
            'test_neg_mean_squared_error': -33.07617655784381,
            'test_r2': 0.42265768153125205,
            'train_neg_mean_squared_error': -24.57523326483063,
            'train_r2': 0.6992062615887933}}
{'_id': ObjectId('6083b75e98e01f8c039f551d'),
 'database': 'boston',
 'date': datetime.datetime(2021, 4, 24, 3, 14, 54, 511000),
 'name': 'SVR',
 'params': {'C': 1.0,
            'cache_size': 200,
            'coef0': 0.0,
 

In [104]:
#Por nome
cond = [{"name" : "RandomForestRegressor"},{ "name" : "Ridge"}]

cursor = db.doc_table.find({"$or" : cond}, {"name" : 1, "scores" : 1}).limit(3)

for doc in cursor:
        pprint(doc)

{'_id': ObjectId('6083bc84779724d204c70a11'),
 'name': 'RandomForestRegressor',
 'scores': {'fit_time': 0.0025064945220947266,
            'score_time': 0.0013309121131896973,
            'test_neg_mean_squared_error': -30.940439204584152,
            'test_r2': 0.36526807869555084,
            'train_neg_mean_squared_error': -9.45903131760828,
            'train_r2': 0.8803711059000352}}
{'_id': ObjectId('6083bc83779724d204c70a10'),
 'name': 'RandomForestRegressor',
 'scores': {'fit_time': 1.478890061378479,
            'score_time': 0.07430434226989746,
            'test_neg_mean_squared_error': -21.667519918362743,
            'test_r2': 0.5797378841842773,
            'train_neg_mean_squared_error': -1.4247502029218828,
            'train_r2': 0.9824477369235751}}
{'_id': ObjectId('6083bc84779724d204c70a12'),
 'name': 'RandomForestRegressor',
 'scores': {'fit_time': 0.0198097825050354,
            'score_time': 0.0020036697387695312,
            'test_neg_mean_squared_error': -26.0

In [124]:
#MAIOR PONTUAÇÃO EM ALGUMA MÉTRICA
#metrica : test_r2

cursor = db.doc_table.find({ "scores.test_r2":{"$ne":""}}, 
                           {"name" : 1, "_id" : 0, "scores" : {"test_r2" : 1}}).limit(1)

cursor = cursor.sort("scores.test_r2", -1)
for doc in cursor:
        pprint(doc)

{'name': 'RandomForestRegressor', 'scores': {'test_r2': 0.6050912044057074}}


In [123]:
#MAIOR PONTUAÇÃO EM ALGUMA MÉTRICA EM ALGUMA BASE
#database : boston
#metrica : test_r2

cursor = db.doc_table.find({ "database": "boston"},{"name" : 1, "_id" : 0,"database": 1, "scores" : {"test_r2" : 1}})

cursor = cursor.sort("scores.test_r2", -1).limit(1)
for doc in cursor:
        pprint(doc)

{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.6050912044057074}}


In [134]:
#RETORNAR OS MODELOS TREINADOS EM UMA DETERMINADA BASE
#database : boston

cursor = db.doc_table.find({ "database": "boston"},
                           {"name" : 1, "_id" : 0,"database": 1, "scores" : 1}).limit(3)
cont = cursor.count()
print (cont)
for doc in cursor:
        pprint(doc)

634
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 0.40048933029174805,
            'score_time': 0.0032573938369750977,
            'test_neg_mean_squared_error': -33.07617655784381,
            'test_r2': 0.42265768153125205,
            'train_neg_mean_squared_error': -24.57523326483063,
            'train_r2': 0.6992062615887933}}
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 1.6131038665771484,
            'score_time': 0.0025395750999450684,
            'test_neg_mean_squared_error': -37.27894737019855,
            'test_r2': 0.25653531407348745,
            'train_neg_mean_squared_error': -23.457981677680237,
            'train_r2': 0.713467488390087}}
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 18.673874735832214,
            'score_time': 0.0035240650177001953,
            'test_neg_mean_squared_error': -38.15119037495376,
            'test_r2': 0.22988459155756136,
            'train_neg_mean_squared_error': -23.605645241107

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [137]:
#RETORNAR OS MODELOS AVALIADOS COM UMA DETERMINADA MÉTRICA (checar se o campo existe) 
#to do

cursor = db.doc_table.find({ "database": "boston"},
                           {"name" : 1, "_id" : 0,"database": 1, "scores" : 1}).limit(3)
cont = cursor.count()
print (cont)

634
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 0.40048933029174805,
            'score_time': 0.0032573938369750977,
            'test_neg_mean_squared_error': -33.07617655784381,
            'test_r2': 0.42265768153125205,
            'train_neg_mean_squared_error': -24.57523326483063,
            'train_r2': 0.6992062615887933}}
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 1.6131038665771484,
            'score_time': 0.0025395750999450684,
            'test_neg_mean_squared_error': -37.27894737019855,
            'test_r2': 0.25653531407348745,
            'train_neg_mean_squared_error': -23.457981677680237,
            'train_r2': 0.713467488390087}}
{'database': 'boston',
 'name': 'SVR',
 'scores': {'fit_time': 18.673874735832214,
            'score_time': 0.0035240650177001953,
            'test_neg_mean_squared_error': -38.15119037495376,
            'test_r2': 0.22988459155756136,
            'train_neg_mean_squared_error': -23.605645241107

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [138]:
#RETORNAR OS MODELOS COM UM DETERMINADO NOME QUE FORAM TREINADOS EM UMA DETERMINADA BASE
#nome : Ridge
#database : boston

cursor = db.doc_table.find({ "database": "boston", "name" : "Ridge"},
                           {"name" : 1, "_id" : 0,"database": 1, "scores" : 1}).limit(3)
cont = cursor.count()
print (cont)

for doc in cursor:
        pprint(doc)

48
{'database': 'boston',
 'name': 'Ridge',
 'scores': {'fit_time': 0.010999202728271484,
            'score_time': 0.0011339783668518066,
            'test_neg_mean_squared_error': -40.75210850730088,
            'test_r2': 0.14851973789321776,
            'train_neg_mean_squared_error': -22.792717532079457,
            'train_r2': 0.7217296061961568}}
{'database': 'boston',
 'name': 'Ridge',
 'scores': {'fit_time': 0.0011656880378723145,
            'score_time': 0.0012121200561523438,
            'test_neg_mean_squared_error': -40.75210850730088,
            'test_r2': 0.14851973789321776,
            'train_neg_mean_squared_error': -22.792717532079457,
            'train_r2': 0.7217296061961568}}
{'database': 'boston',
 'name': 'Ridge',
 'scores': {'fit_time': 0.0012112855911254883,
            'score_time': 0.0010199546813964844,
            'test_neg_mean_squared_error': -42.362299330127634,
            'test_r2': 0.11415180142489323,
            'train_neg_mean_squared_error': -

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


In [160]:
#ENCONTRAR MODELOS COM ALGUMA MÉTRICA EM UM INTERVALO ESPECÍFICO
#metrica : test_r2
#intervalo : 0.3 < x < 0.5


cursor = db.doc_table.find({ "$and" : [{"scores.test_r2":{"$lt":0.5}},{"scores.test_r2":{"$gt":0.3}}]},
                           {"name" : 1, "_id" : 0,"database": 1, "scores" : {"test_r2" : 1}})

#cursor = cursor.sort("scores.test_r2", -1).limit(1)
for doc in cursor:
        pprint(doc)


{'database': 'boston',
 'name': 'SVR',
 'scores': {'test_r2': 0.42265768153125205}}
{'database': 'diabetes',
 'name': 'SVR',
 'scores': {'test_r2': 0.43983215751749444}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.36526807869555084}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.3627544928658441}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4187578441307834}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.3601357240868527}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.40361383834821796}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.367484203925967}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.30935241962808413}}
{'database': 'boston',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.48688926798334053}}
{'database': 'bosto

{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.42647847503452363}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4087586020080548}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.43200266547456806}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4447048075423924}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4065537914960324}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.39514430914182974}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.44141689350956803}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.43967266748217265}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4430821788278493}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores'

{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4288121369139887}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4338108458826793}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4393940295533041}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.39661820058920344}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.43411183302323314}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4111630709911185}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4152378025206268}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4331583930796822}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4362108830451287}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {

 'scores': {'test_r2': 0.41769947439530963}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4146925831095938}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.42217874292224167}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.42523658656649255}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4357941211093551}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.42862009958445435}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4251273363188782}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.40007402343206677}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.4086704494975656}}
{'database': 'diabetes',
 'name': 'RandomForestRegressor',
 'scores': {'test_r2': 0.42841822697012877}}
{'database': 'diabetes'

In [ ]:
#CONSULTAR POR DATA
#data : 

In [166]:
#RETORNAR QUEM TEM UMA METRICA DE TESTE MAIOR QUE A DE TREINO to do *agregation
#metrica : test_r2, train_r2

cursor = db.doc_table.find({"scores.test_r2":{"$gt":"scores.train_r2"}},
                           {"name" : 1, "_id" : 0,"database": 1, "scores" : {"test_r2" : 1}})

#cursor = cursor.sort("scores.test_r2", -1).limit(1)
for doc in cursor:
        pprint(doc)


In [ ]:
#agregaçao

db.collection.agregate(<stage1><stage2><stage3>)
$group juntar
p

## Complexos

In [191]:
#CONTAR QUANTOS MODELOS DE CADA TIPO EXISTEM

cursor = db.doc_table.aggregate([{"$group" : {"_id":"$name", "total": {"$sum": 1}}}])

for doc in cursor:
        pprint(doc)

{'_id': 'RandomForestRegressor', 'total': 1152}
{'_id': 'Ridge', 'total': 96}
{'_id': 'RandomForestClassifier', 'total': 1152}
{'_id': 'SVR', 'total': 20}
{'_id': 'LogisticRegression', 'total': 69}


In [186]:
#CONTAR QUANTOS MODELOS FORAM TREINADOS EM UMA BASE
#database : boston

cursor = db.doc_table.aggregate([{"$match": {"database": "boston"}},
                                 {"$group" : {"_id":"$database", "total": {"$sum": 1}}}])

for doc in cursor:
        pprint(doc)

{'_id': 'boston', 'total': 634}


In [223]:
#RETORNAR O MODELO COM MAIOR PONTUAÇÃO MÉDIA EM UMA MÉTRICA
#metrica : test_r2

cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$exists": True}}},
                                  {"$group" : {"_id":"$name", "avgAmount": {"$avg":"$scores.test_r2"}}}])

for doc in cursor:
        pprint(doc)

{'_id': 'SVR', 'avgAmount': -1559803.8107531858}
{'_id': 'RandomForestRegressor', 'avgAmount': 0.4060676304783088}
{'_id': 'Ridge', 'avgAmount': -0.9006692195839653}


In [224]:
#RETIRANDO VALORES NEGATIVOS

cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$gt": 0}}},
                                 {"$match": {"scores.test_r2": {"$exists": True}}},
                                  {"$group" : {"_id":"$name", "avgAmount": {"$avg":"$scores.test_r2"}}},
                                {"$limit":1}])

for doc in cursor:
        pprint(doc)

{'_id': 'SVR', 'avgAmount': 0.29102535883886943}


In [240]:
#ENCONTRAR O VALOR MÉDIO, MIN, MAX E O DESVIO PADRÃO DE ALGUMA MÉTRICA
#metrica : test_r2

cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$gt": 0}}},
                                 {"$match": {"scores.test_r2": {"$exists": True}}},
                                 {"$group" :{"_id":"total", "avgAmount": {"$avg":"$scores.test_r2"}, 
                                              "max": {"$max":"$scores.test_r2"},
                                             "min": {"$min":"$scores.test_r2"},
                                              "std": {"$stdDevPop":"$scores.test_r2"}}},
                                {"$project":{"avgAmount": 1, "max": 1, "min": 1, "std":1}}])

for doc in cursor:
        pprint(doc)

{'_id': 'total',
 'avgAmount': 0.418166717282541,
 'max': 0.6050912044057074,
 'min': 0.0015640829004583967,
 'std': 0.14363882582420429}


In [241]:
#ENCONTRAR O VALOR MÉDIO, MIN, MAX E O DESVIO PADRÃO DE ALGUMA MÉTRICA POR MODELO
#metrica : test_r2

cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$gt": 0}}},
                                 {"$match": {"scores.test_r2": {"$exists": True}}},
                                 {"$group" : {"_id":"$name", "avgAmount": {"$avg":"$scores.test_r2"}, 
                                              "max": {"$max":"$scores.test_r2"},
                                             "min": {"$min":"$scores.test_r2"},
                                             "std": {"$stdDevPop":"$scores.test_r2"}}}])

for doc in cursor:
        pprint(doc)

{'_id': 'SVR',
 'avgAmount': 0.29102535883886943,
 'max': 0.43983215751749444,
 'min': 0.11703293001346693,
 'std': 0.111645746957036}
{'_id': 'RandomForestRegressor',
 'avgAmount': 0.42727512260276934,
 'max': 0.6050912044057074,
 'min': 0.0015640829004583967,
 'std': 0.13880995694305207}
{'_id': 'Ridge',
 'avgAmount': 0.24422183244193776,
 'max': 0.4848910834190733,
 'min': 0.11415180142489323,
 'std': 0.12623712883127497}


In [242]:
#ENCONTRAR O VALOR MÉDIO, MIN, MAX E O DESVIO PADRÃO DE ALGUMA MÉTRICA POR MODELO E BASE DE DADOS
#metrica : test_r2

cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$gt": 0}}},
                                 {"$match": {"scores.test_r2": {"$exists": True}}},
                                 {"$group" : {"_id":["$database","$name"], "avgAmount": {"$avg":"$scores.test_r2"}, 
                                              "max": {"$max":"$scores.test_r2"},
                                             "min": {"$min":"$scores.test_r2"},
                                             "std": {"$stdDevPop":"$scores.test_r2"}}}])

for doc in cursor:
        pprint(doc)

{'_id': ['boston', 'Ridge'],
 'avgAmount': 0.18709753132865953,
 'max': 0.33885933787655603,
 'min': 0.11415180142489323,
 'std': 0.05622025623484519}
{'_id': ['boston', 'SVR'],
 'avgAmount': 0.2973217663755638,
 'max': 0.42265768153125205,
 'min': 0.22988459155756136,
 'std': 0.07452051608206783}
{'_id': ['boston', 'RandomForestRegressor'],
 'avgAmount': 0.49456093192070055,
 'max': 0.6050912044057074,
 'min': 0.01843598889152609,
 'std': 0.10947451071261617}
{'_id': ['diabetes', 'Ridge'],
 'avgAmount': 0.37989204758597367,
 'max': 0.4848910834190733,
 'min': 0.12249413251560287,
 'std': 0.14184565898157736}
{'_id': ['diabetes', 'SVR'],
 'avgAmount': 0.27843254376548066,
 'max': 0.43983215751749444,
 'min': 0.11703293001346693,
 'std': 0.16139961375201378}
{'_id': ['diabetes', 'RandomForestRegressor'],
 'avgAmount': 0.357406128649232,
 'max': 0.4447048075423924,
 'min': 0.0015640829004583967,
 'std': 0.1313335278281588}


In [243]:
#AGRUPAR OS MODELOS POR BASE DE DADOS E ENCONTRAR O VALOR MÉDIO DE CADA UMA DAS MÉTRICAS NESSAS BASES

#TO DO


cursor = db.doc_table.aggregate([{"$match": {"scores.test_r2": {"$gt": 0}}},
                                 {"$match": {"scores.test_r2": {"$exists": True}}},
                                 {"$group" : {"_id":["$database"], "avgAmount": {"$avg":"$scores.test_r2"}, 
                                              "max": {"$max":"$scores.test_r2"},
                                             "min": {"$min":"$scores.test_r2"},
                                             "std": {"$stdDevPop":"$scores.test_r2"}}}])

for doc in cursor:
        pprint(doc)

{'_id': ['boston'],
 'avgAmount': None,
 'max': 0.6050912044057074,
 'min': 0.01843598889152609,
 'std': 0.13083892700135313}
{'_id': ['diabetes'],
 'avgAmount': None,
 'max': 0.4848910834190733,
 'min': 0.0015640829004583967,
 'std': 0.13189860147335128}


In [ ]:
#AGRUPAR OS MODELOS POR BASE DE DADOS E ENCONTRAR O MELHOR MODELO EM CADA UMA DELAS (essa é difícil porquê cada modelo pode ser avaliado com uma métrica diferente)
